In [ ]:
#COLAB
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:

#STYLEGAN2 MODEL TRAINING - DATABASE ALL PART1 RANDOMLY CROPPED
#by JON THUM


!git clone https://github.com/jonthum/AiPainter

#COLAB ROOT DIRECTORY
import sys
ROOT_DIR = '/content/AiPainter/lib/stylegan2/'
sys.path.append(ROOT_DIR)

#OLD TENSORFLOW VERSION
%tensorflow_version 1.x
import tensorflow as tf
print('Tensorflow version: {}'.format(tf.__version__) )

#IMPORT DATASET TOOLS
from dataset_tool import TFRecordExporter
import numpy as np
import random
import json
from PIL import Image
from PIL import ImageFilter

#CHECK CUDA
!nvidia-smi -L


In [ ]:
#TRANSFER 'All Part1' DB
!gdown --id 1nuRVMGKxxr3kOpuOO6bT2FXIgUuE0IOB
!tar -xf all_part1.tgz

#TRANSFER LATEST MODEL network-snapshot-003342.pkl
!gdown --id  12K_gzHc8jQgyCL3YRZqCHmXtddU2XovP

#TRANSFER JSON FILE
!gdown --id  1Iq7wrumEvdx7ev9_G3FAKzQicGB_9SSx




In [ ]:
#LOAD FILES
with open('filenames_all_part1.json') as json_file:
    files = json.load(json_file)
count = len(files)
print(count)

def rand_crop(img):
    zoom = random.uniform(1, 1.15)
    size = 1024/zoom
    
    mid = random.uniform(1,3)
    std = random.uniform(2,6)
    
    randx = np.random.normal((1024-size)/mid,(1024-size)/std)
    randx = int(np.clip(randx,0,(1024-size)))
    
    randy = np.random.normal((1024-size)/mid,(1024-size)/std)
    randy = int(np.clip(randy,0,(1024-size)))
    
    crop = img.crop((randx, randy, randx+size, randy+size))
    crop = crop.filter(ImageFilter.DETAIL)
    
    img = crop.resize((1024, 1024), Image.LANCZOS)
    
    return img

    

In [ ]:
#MAKE TENSORFLOW RECORDS

tfrecord_dir = '/content/datasets/all_part1'

with TFRecordExporter(tfrecord_dir, count) as tfr:
    order = tfr.choose_shuffled_order()
    for idx in range(order.size):
        fn = files[idx]
        img = Image.open(fn)
        img = rand_crop(img)
        img = np.transpose(np.array(img), (2, 0, 1))
        tfr.add_image(np.array(img))



In [ ]:
%cd '/content/AiPainter/lib/stylegan2/'

#TRAIN - To speed up training change metric to 'None'
!python run_training.py --num-gpus=1 --data-dir='/content/datasets/' --config=config-f \
--dataset=all_part1  --min-h=4 --min-w=4 --res-log2=8 --metric=fid50k \
--mirror-augment=True --mirror-augment-v=False --total-kimg=26000 \
--result-dir='/content/AiPainter/custom/training/data/all_rand1' \
--resume-pkl '/content/network-snapshot-003342.pkl' #'latest'
